### Prediction of store sales (Task-2)

In [1]:
### import Libraries 
import tensorflow as tf

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import numpy as np
from statsmodels.tsa.stattools import adfuller, acf, pacf
import pandas as pd
from sklearn.preprocessing import Normalizer, MinMaxScaler, StandardScaler, RobustScaler
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
sns.set()


In [33]:
### Import scripts & modules 
import os
import sys
sys.path.append(os.path.abspath(os.path.join(r'E:/KAIM 2/KAIM 2 Week 4/Week4/scripts')))

from helper import MyHelper
import utility
import plots
Helper = MyHelper()
#from scripts.helper import percent_missing

2.1 Preprocessing

In [4]:
### Load the data
train_PATH = "../data/train.csv"
test_PATH = "../data/test.csv"
store_PATH = "../data/store.csv"
sample_PATH = "../data/sample_submission.csv"

In [5]:
print(type[test_PATH])

type['../data/test.csv']


In [6]:
# Load the CSV file into a DataFrame
train = pd.read_csv(train_PATH)

# Now you can call describe on the DataFrame
statistics = train.describe(include="all")
print(statistics)

C:\Users\you\AppData\Local\Temp\ipykernel_13572\2234050922.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(train_PATH)


               Store     DayOfWeek         Sales     Customers          Open  \
count   1.017209e+06  1.017209e+06  1.017209e+06  1.017209e+06  1.017209e+06   
unique           NaN           NaN           NaN           NaN           NaN   
top              NaN           NaN           NaN           NaN           NaN   
freq             NaN           NaN           NaN           NaN           NaN   
mean    5.584297e+02  3.998341e+00  5.472856e+03  5.783493e+02  8.301067e-01   
std     3.219087e+02  1.997391e+00  3.323989e+03  3.483565e+02  3.755392e-01   
min     1.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
25%     2.800000e+02  2.000000e+00  3.727000e+03  4.050000e+02  1.000000e+00   
50%     5.580000e+02  4.000000e+00  5.744000e+03  6.090000e+02  1.000000e+00   
75%     8.380000e+02  6.000000e+00  7.584000e+03  7.940000e+02  1.000000e+00   
max     1.115000e+03  7.000000e+00  1.404900e+04  1.485000e+03  1.000000e+00   

               Promo StateHoliday  Scho

In [8]:
#test_PATH.describe(include="all")
# Load the CSV file into a DataFrame
test = pd.read_csv(test_PATH)

# Now you can call describe on the DataFrame
statistics = test.describe(include="all")
print(statistics)

                  Id         Store     DayOfWeek        Date          Open  \
count   41088.000000  41088.000000  41088.000000       41088  41077.000000   
unique           NaN           NaN           NaN          48           NaN   
top              NaN           NaN           NaN  2015-09-17           NaN   
freq             NaN           NaN           NaN         856           NaN   
mean    20544.500000    555.899533      3.979167         NaN      0.854322   
std     11861.228267    320.274496      2.015481         NaN      0.352787   
min         1.000000      1.000000      1.000000         NaN      0.000000   
25%     10272.750000    279.750000      2.000000         NaN      1.000000   
50%     20544.500000    553.500000      4.000000         NaN      1.000000   
75%     30816.250000    832.250000      6.000000         NaN      1.000000   
max     41088.000000   1115.000000      7.000000         NaN      1.000000   

               Promo StateHoliday  SchoolHoliday  
count   4108

In [9]:
# the class has utility function we are going to use
Helper = MyHelper()

In [10]:
# we use our helper function to read csv data
train = Helper.read_csv(train_PATH)
test = Helper.read_csv(test_PATH)

E:\KAIM 2\KAIM 2 Week 4\Week4\scripts\helper.py:12: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


>> file read as csv
>> file read as csv


In [11]:
print(Helper.read_csv)

<bound method MyHelper.read_csv of <helper.MyHelper object at 0x00000272B9159D30>>


In [12]:
Helper.percent_missing(train)

C:\Users\you\AppData\Local\Temp\ipykernel_13572\3751687797.py:1: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  Helper.percent_missing(train)


0.77

In [13]:
def get_feature(df):
    return df[["Dates", "Sales"]]

In [14]:
train.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,...,Dates,StoreType,Assortment,CompetitionDistance,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,SalesperCustomer,CompetitionOpenSince
0,1,5,5263.0,555.0,1,1,0,1,2015,7,...,2015-07-31,c,a,1270.0,0,0.0,0.0,0,9.482883,82.0
1,2,5,6064.0,625.0,1,1,0,1,2015,7,...,2015-07-31,a,a,570.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",9.702400,92.0
2,3,5,8314.0,821.0,1,1,0,1,2015,7,...,2015-07-31,a,a,14130.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",10.126675,103.0
3,4,5,13995.0,609.0,1,1,0,1,2015,7,...,2015-07-31,c,c,620.0,0,0.0,0.0,0,22.980296,70.0
4,5,5,4822.0,559.0,1,1,0,1,2015,7,...,2015-07-31,a,a,29910.0,0,0.0,0.0,0,8.626118,3.0


In [15]:
def drop_closed(df):
    df = df[df['Open'] == 1]
    return df

In [16]:
def scaler(df, columns, mode="minmax"):
    if (mode == "minmax"):
        minmax_scaler = MinMaxScaler()
        return pd.DataFrame(minmax_scaler.fit_transform(df), columns=columns), minmax_scaler
    elif (mode == "standard"):
        scaler = StandardScaler()
        return pd.DataFrame(scaler.fit_transform(df), columns=columns), scaler
    elif (mode == "robust"):
        scaler = RobustScaler()
        return pd.DataFrame(scaler.fit_transform(df), columns=columns), scaler

In [17]:
def add_scaled_sales(df):
    scaled_sales, scaler_obj = scaler(df[["Sales"]], mode="minmax", columns=["scaled_sales"])
    df["scaled_sales"] = scaled_sales["scaled_sales"].to_list()
    return df, scaler_obj

In [18]:
def handle_outliers(df, col, method="lower_upper"):
        
        df = df.copy()
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)

        lower_bound = q1 - ((1.5) * (q3 - q1))
        upper_bound = q3 + ((1.5) * (q3 - q1))

        if method == "mean":
            df[col] = np.where(df[col] < lower_bound,   df[col].mean(), df[col])
            df[col] = np.where(df[col] > upper_bound, df[col].mean(), df[col])

        elif  method == "mode":
            df[col] = np.where(df[col] < lower_bound,   df[col].mode()[0], df[col])
            df[col] = np.where(df[col] > upper_bound, df[col].mode()[0], df[col])
        else:
            df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
            df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])

        return df


In [19]:
print(type[train_PATH])

type['../data/train.csv']


In [20]:
train["Sales"] = train["Sales"].astype("float")

In [21]:
print(train.columns)

Index(['Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Day', 'WeekOfYear',
       'Dates', 'StoreType', 'Assortment', 'CompetitionDistance', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval',
       'SalesperCustomer', 'CompetitionOpenSince'],
      dtype='object')


In [22]:
data = drop_closed(train)
data = get_feature(data)
data = handle_outliers(data, "Sales")

In [23]:
import helper
print(dir(helper))  # This will list all attributes of the helper module

['MinMaxScaler', 'MyHelper', 'Normalizer', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__warningregistry__', 'np', 'pd']


In [45]:
# Make sure helper.py is in the same directory or adjust the path accordingly

from helper import percent_missing

#sys.path.append(os.path.abspath(os.path.join('E:/KAIM 2/KAIM 2 Week 4/Week4/scripts')))

# Assuming 'data' is already defined and contains the relevant columns
data_agg = data.groupby("Dates").agg({"Sales": "mean"}).reset_index()

# Check for percentage of missing values
percent_missing(data_agg)

# Get descriptive statistics
print(data_agg.describe())

ImportError: cannot import name 'percent_missing' from 'helper' (E:\KAIM 2\KAIM 2 Week 4\Week4\scripts\helper.py)

In [44]:
 data_agg = data.groupby("Dates").agg({"Sales": "mean"}).reset_index()
 helper.percent_missing(data_agg)
 data_agg.describe()
# import pandas as pd
# import helper  # Ensure you have imported the helper module

# Assuming 'data' is already defined and contains the relevant columns
#data_agg = data.groupby("Dates").agg({"Sales": "mean"}).reset_index()

# Check for percentage of missing values
#helper.percent_missing(data_agg)

# Get descriptive statistics
#print(data_agg.describe())

AttributeError: module 'helper' has no attribute 'percent_missing'

In [25]:
display(train.head())
display(train.describe())
display(train.info())

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,...,Dates,StoreType,Assortment,CompetitionDistance,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,SalesperCustomer,CompetitionOpenSince
0,1,5,5263.0,555.0,1,1,0,1,2015,7,...,2015-07-31,c,a,1270.0,0,0.0,0.0,0,9.482883,82.0
1,2,5,6064.0,625.0,1,1,0,1,2015,7,...,2015-07-31,a,a,570.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",9.702400,92.0
2,3,5,8314.0,821.0,1,1,0,1,2015,7,...,2015-07-31,a,a,14130.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",10.126675,103.0
3,4,5,13995.0,609.0,1,1,0,1,2015,7,...,2015-07-31,c,c,620.0,0,0.0,0.0,0,22.980296,70.0
4,5,5,4822.0,559.0,1,1,0,1,2015,7,...,2015-07-31,a,a,29910.0,0,0.0,0.0,0,8.626118,3.0


,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,Year,Month,Day,WeekOfYear,CompetitionDistance,Promo2,Promo2SinceWeek,Promo2SinceYear,SalesperCustomer,CompetitionOpenSince
count,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,844340.000000,1.017209e+06
mean,5.584297e+02,3.998341e+00,5.472856e+03,5.783493e+02,8.301067e-01,3.815145e-01,1.786467e-01,2.013832e+03,5.846762e+00,1.570279e+01,2.361551e+01,5.422034e+03,5.005638e-01,1.164767e+01,1.007011e+03,9.722615,4.114934e+01
std,3.219087e+02,1.997391e+00,3.323989e+03,3.483565e+02,3.755392e-01,4.857586e-01,3.830564e-01,7.773960e-01,3.326097e+00,8.787638e+00,1.443338e+01,7.706913e+03,4.999999e-01,1.532393e+01,1.005877e+03,2.550463,6.601545e+01
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.013000e+03,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-3.100000e+01
25%,2.800000e+02,2.000000e+00,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,0.000000e+00,2.013000e+03,3.000000e+00,8.000000e+00,1.100000e+01,7.100000e+02,0.000000e+00,0.000000e+00,0.000000e+00,8.007077,0.000000e+00
50%,5.580000e+02,4.000000e+00,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,0.000000e+00,2.014000e+03,6.000000e+00,1.600000e+01,2.200000e+01,2.330000e+03,1.000000e+00,1.000000e+00,2.009000e+03,9.431414,1.600000e+01
75%,8.380000e+02,6.000000e+00,7.584000e+03,7.940000e+02,1.000000e+00,1.000000e+00,0.000000e+00,2.014000e+03,8.000000e+00,2.300000e+01,3.500000e+01,6.880000e+03,1.000000e+00,2.200000e+01,2.012000e+03,11.005882,7.300000e+01
max,1.115000e+03,7.000000e+00,1.404900e+04,1.485000e+03,1.000000e+00,1.000000e+00,1.000000e+00,2.015000e+03,1.200000e+01,3.100000e+01,5.200000e+01,7.586000e+04,1.000000e+00,5.000000e+01,2.015000e+03,51.694444,1.386000e+03


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 22 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   Store                 1017209 non-null  int64  
 1   DayOfWeek             1017209 non-null  int64  
 2   Sales                 1017209 non-null  float64
 3   Customers             1017209 non-null  float64
 4   Open                  1017209 non-null  int64  
 5   Promo                 1017209 non-null  int64  
 6   StateHoliday          1017209 non-null  object 
 7   SchoolHoliday         1017209 non-null  int64  
 8   Year                  1017209 non-null  int64  
 9   Month                 1017209 non-null  int64  
 10  Day                   1017209 non-null  int64  
 11  WeekOfYear            1017209 non-null  int64  
 12  Dates                 1017209 non-null  object 
 13  StoreType             1017209 non-null  object 
 14  Assortment            1017209 non-

None